In [10]:
from time import time
import numpy as np
import pandas as pd

import cuml; print('cuML '+cuml.__version__)
import sklearn; print('skLearn '+sklearn.__version__)

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from cuml.ensemble import RandomForestRegressor as RFGPU

cuML 0.15.0
skLearn 0.23.2


In [11]:
df = pd.read_csv('samples.csv', dtype='float32')
df.head()

,height,age,frost,soil,aspect,elev,slope,bio01,bio02,bio03,...,bio10,bio11,bio12,bio13,bio14,bio15,bio16,bio17,bio18,bio19
0,0.763126,72.0,0.043042,818.0,211.250870,414.429260,0.656142,-2.354167,8.825000,20.764706,...,12.016666,-18.100000,1030.0,117.0,59.0,21.739109,325.0,198.0,318.0,208.0
1,0.926581,58.0,0.050841,2559.0,207.564056,457.275330,0.946029,0.420833,10.841667,22.824562,...,15.916667,-17.033333,445.0,85.0,14.0,60.497139,210.0,52.0,210.0,56.0
2,2.285333,1.0,0.486664,1194.0,100.369385,404.196289,0.149261,-2.516667,12.650000,24.950691,...,14.100000,-20.016666,409.0,73.0,18.0,54.793449,190.0,54.0,190.0,58.0
3,2.062481,33.0,0.000000,296.0,220.769760,262.464081,0.444754,0.016667,7.850000,21.302588,...,12.441667,-13.566667,1135.0,116.5,63.0,17.656145,335.0,220.5,311.5,242.0
4,3.707132,54.0,0.002849,2449.0,257.453705,436.415527,0.456665,0.150000,12.116667,25.835110,...,14.666667,-16.183332,428.0,77.0,17.0,55.434898,197.0,54.0,197.0,65.0


In [12]:
test_ratio = 0.2
df_train, df_test = train_test_split(df, test_size=test_ratio, random_state=43)
label_name = 'height'
X_train, y_train = df_train.drop(label_name, axis=1), df_train[label_name]
X_test, y_test = df_test.drop(label_name, axis=1), df_test[label_name]

In [13]:
## sk-learn regressor
start = time()
sk_regr = RandomForestRegressor(n_estimators=100,
                                max_depth=25,
                                n_jobs=-1,
                                random_state=42)
sk_regr.fit(X_train, y_train)
pred = sk_regr.predict(X_train)
end = time()

print(f"The R^2 score is: {np.round(r2_score(pred, y_train), 2)} and it took {np.round(end - start, 2)} seconds")

The R^2 score is: 0.83 and it took 3.96 seconds


In [14]:
## cuML regressor
start = time()
cu_regr = RFGPU(n_estimators=100,
                max_depth=25) 
                #n_bins=64) 
                #,split_algo=1)
cu_regr.fit(X_train, y_train)
pred = cu_regr.predict(X_train)
end = time()

print(f"The R^2 score is: {np.round(r2_score(pred, y_train), 2)} and it took {np.round(end - start, 2)} seconds")

The R^2 score is: 0.4 and it took 4.74 seconds
